In [1]:
#imports
import torch
import torch.nn as nn
from torch.nn import functional as F

#switches device to cuda (gpu) and detects if switch was successful
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Current Device: " + device)

#declare constants
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

Current Device: cuda


In [23]:
#opens the training data with the correct encoding
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#gets the llm alphabet
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [24]:
#encoder and decoder for the alphabet
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encodes the training data
data = torch.tensor(encode(text), dtype=torch.long)

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [25]:
#sets training data to 80% of the text and allocates the rest for validation
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    #sets the train and val split for the batch
    data = train_data if split == 'train' else val_data
    
    #gets a batch of random indices
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    #sets "batch" number of input and target vectors of size "block_size"
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1:i + block_size + 1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

''' Check get_batch()

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)
'''

tensor([165877, 147308,  19809,  14630])
inputs:
tensor([[79, 11,  3,  0,  0,  3, 28, 68],
        [74, 67, 57,  1, 62, 73, 72,  1],
        [68, 75, 58, 71,  1, 72, 74, 56],
        [78,  1, 66, 68, 71, 58,  1, 59]], device='cuda:0')
targets:
tensor([[11,  3,  0,  0,  3, 28, 68, 67],
        [67, 57,  1, 62, 73, 72,  1, 58],
        [75, 58, 71,  1, 72, 74, 56, 61],
        [ 1, 66, 68, 71, 58,  1, 59, 71]], device='cuda:0')


In [26]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

''' show what program sees as input and shows target

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, ' target is ', target)
'''

when input is  tensor([80])  target is  tensor(1)
when input is  tensor([80,  1])  target is  tensor(1)
when input is  tensor([80,  1,  1])  target is  tensor(28)
when input is  tensor([80,  1,  1, 28])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39])  target is  tensor(42)
when input is  tensor([80,  1,  1, 28, 39, 42])  target is  tensor(39)
when input is  tensor([80,  1,  1, 28, 39, 42, 39])  target is  tensor(44)
when input is  tensor([80,  1,  1, 28, 39, 42, 39, 44])  target is  tensor(32)


In [29]:
class BigramLanguageModel(nn.Module):
    #llm constructor
    def __init__(self, vocab_size):
        super().__init__()
        #create an embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    # forward pass method that takes input of "index" and "target" and computes predictions for the next token in the sequence
    def forward(self, index, targets=None):
        #creates logits (unnormalized predictions) from the input text and the embedding table
        logits = self.token_embedding_table(index)
        
        #loss calculations
        if targets is None:
            #sets loss to none if no targets are input
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #calculates loss of generated text based on targets
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    #generates "max_new_tokens" number of new tokens based on the input "index"
    def generate(self, index, max_new_tokens):
        #generates new tokens one at a time
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            #gets the new token in the sequence
            logits = logits[:,-1,:]
            #generates and samples probibility distribution for next character and appends it to index
            probs = F.softmax(logits, dim = -1)
            index_next = torch.multinomial(probs, num_samples = 1)
            index = torch.cat((index, index_next), dim = 1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


wMMtQmNb&
&8!:fuZWeOG33_R-!I﻿N0bNAZanO:,4&,T2Gm'qaF(cebjj*zjeSqJ;i,xhmxhg'Q*;xgnx'Wy ngE:7Z3i_'It?6'Sulihmn*B25z;9Y5f
AQ0SmBfY6U(?aro8TnxCXtW
zB]F(VcA0:u﻿e
ZhXJ,je5;TA&"1fzSxcq
!I3t TGZae
t6XmN6'L(J,A52;LOyEjvN2:Epo1:G:3e
*wRMt4UU(:9ZaeMtDTax8D]M"*?4bcRSv﻿Pcego2TF''1D6WO:!'xhK-,cMtCp5&1hkxn
,9t PItd
Y*BhX1bivfvu33tvyimaphAPbx,q'8_RMk2I
_tcT.b)Db TRG,A[g T&''99?L56n2MqS" HL;hO:wjn PmSJhN2mu3YjyFK'583t*.Gaxh06ekjDQL7Vn﻿.Ho CTcTLhcZ R0reMkhPXj(hv;Jf9bvVr ",Ab "GNup5XIj''1?OrhCBo8XLhg
S!jvkcBMLn(dIj


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())